In [ ]:
import pandas as pd
from transformers import pipeline
from tqdm import tqdm

In [24]:
# Load the dataset
file_path = 'financialNews.csv'
df = pd.read_csv(file_path)

In [25]:
# Step 1: Remove duplicate articles
df = df.drop_duplicates(subset='description', keep='first')

In [26]:
# Step 2: Filter descriptions
df = df[~df['description'].str.startswith("By")]  # Exclude rows where description starts with "By"
df = df[~df['description'].str.startswith("(Update")]
df = df[df['description'].str.len() >=30]  # Exclude rows with descriptions shorter than 40 characters

In [28]:
# Step 3: Reset index for easier processing
df.reset_index(drop=True, inplace=True)

In [29]:
# Step 4: Initialize FinBERT sentiment analysis pipeline
finbert = pipeline("sentiment-analysis", model="yiyanghkust/finbert-tone", tokenizer="yiyanghkust/finbert-tone")

config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

c:\Users\astev\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\astev\.cache\huggingface\hub\models--yiyanghkust--finbert-tone. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tf_model.h5:   0%|          | 0.00/439M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at yiyanghkust/finbert-tone.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

In [ ]:
# Step 5: Apply sentiment analysis in batches
def classify_sentiment_batch(descriptions):
    results = finbert(list(descriptions))  # Process batch as a list
    return [result['label'] for result in results]

In [ ]:
# Set batch size
batch_size = 16  # Adjust based on memory availability and performance

In [ ]:
# Apply batch processing
sentiments = []
for i in tqdm(range(0, len(df), batch_size), desc="Processing Batches"):
    batch = df['description'][i:i+batch_size]
    sentiments.extend(classify_sentiment_batch(batch))

KeyboardInterrupt: 

In [ ]:
# Add sentiment results to the dataframe
df['sentiment'] = sentiments

In [ ]:
# Step 6: Save the results
output_file = 'sentiment_analysis_results.csv'
df.to_csv(output_file, index=False)

print(f"Sentiment analysis completed. Results saved to {output_file}")